In [6]:
from factorizer_wrappers import ICA_Factorizer, NMF_Factorizer, PCA_Factorizer
from factorizer_wrappers import example_V, test_example_V, test_Factorizer

In [9]:
test_example_V()

test_Factorizer(ICA_Factorizer(n_components=4), atol=0.5) 
test_Factorizer(ICA_Factorizer(n_components=5), atol=0.001) 

test_Factorizer(NMF_Factorizer(n_components=4), atol=0.5) 
test_Factorizer(NMF_Factorizer(n_components=5), atol=0.001) 

test_Factorizer(PCA_Factorizer(n_components=4), atol=0.5) 
test_Factorizer(PCA_Factorizer(n_components=5), atol=0.001) 

test_example_V() passed.
ICA_Factorizer(n_components=4)
test_Factorizer (ICA_Factorizer) passed
ICA_Factorizer(n_components=5)
test_Factorizer (ICA_Factorizer) passed
NMF_Factorizer(n_components=4)
test_Factorizer (NMF_Factorizer) passed
NMF_Factorizer(n_components=5)
test_Factorizer (NMF_Factorizer) passed
PCA_Factorizer(n_components=4)
test_Factorizer (PCA_Factorizer) passed
PCA_Factorizer(n_components=5)
test_Factorizer (PCA_Factorizer) passed
